BoxPlots for All patients

In [6]:
#Import Modules

import sys
sys.path.append("..") #give this script access to all modules in parent directory
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import matplotlib.lines as mlines
from scipy.io import loadmat
import scipy
from scipy.stats import skew
from Data_Cleaning import preprocess
from Data_Cleaning import preprocess_sickbay
import Filtering
import Actigraph_Metrics

Define Functions

Load All Patient Data

In [7]:
data_dir = r'C:\Users\sidha\OneDrive\Sid Stuff\PROJECTS\iMEDS Design Team\Data Analysis\PedAccel\data_analysis\PythonPipeline\PatientData'

# data_dir = r'C:\Users\jakes\Documents\DT 6 Analysis\PythonCode\PedAccel\data_analysis\PythonPipeline\PatientData'
window_size = 100 #100 is 1 second worth of time
lead_time = 10
slice_size_min = 15
sr = .5


In [8]:
def fillArrays(signal, Mean, AUC, PEAK_PEAK): 
    x = []
    for i in range(len(signal)):
        x.append(i) #x is the x axis for signal data, it is necessary for AUC calculations. Signal is ordered as is x, so this works
    Mean.append(np.mean(signal))
    AUC.append(Actigraph_Metrics.calc_area_under_curve(x,signal))
    PEAK_PEAK.append(max(signal)-min(signal))

def fill_SBS_Changes(vitals_SBS,SBS_Changes,i):
    state = vitals_SBS[i+1]-vitals_SBS[i]
    if(state > 0):
        SBS_Changes.append(1) #1 signifies increase in SBS
    elif state == 0: 
        SBS_Changes.append(0)
    else:
        SBS_Changes.append(-1)


In [5]:
#There is no error handling in place, the .mat file must exist
for patient in os.listdir(data_dir):
    # filter out non-directories
    patient_dir = os.path.join(data_dir, patient)
    if os.path.isdir(patient_dir):
        # data_filepath_accel = os.path.join(patient_dir, f'{patient}_{lead_time}MIN_{slice_size_min - lead_time}MIN.mat')           
        data_filepath_vitals = os.path.join(patient_dir, f'{patient}_SICKBAY{lead_time}MIN_{slice_size_min - lead_time}MIN.mat')   
        # data_filepath_vitals = os.path.join(patient_dir, f'{patient}_SICKBAY_{slice_size_min - lead_time}MIN_{lead_time}MIN.mat')
        
       # accel_data = loadmat(data_filepath_accel)
       # x_mag = accel_data["x_mag"]
       # accel_SBS = accel_data["sbs"].flatten()
        
        vitals_data = loadmat(data_filepath_vitals)
        temp_hr = vitals_data['heart_rate']
        temp_SpO2 = vitals_data['SpO2']
        temp_rr = vitals_data['respiratory_rate']

        vitals_SBS = vitals_data['sbs'].flatten()
        hr = []
        rr = []
        SpO2 = []
        bpm = []
        bps = []
        bpd = []
        vitals_list = [hr,rr,SpO2]
        vitals_names = ['hr','rr','spo2']
        temp_vitals = [temp_hr,temp_rr, temp_SpO2] 
        
        flag_list = [0] * (int)(sr * 60 * slice_size_min) #generate a list to insert in place of invalid data, 
        #this list serves as a flag for a window to ignore in the box plot function

        for j in range(len(vitals_list)): #go through every vitals metric
            print(f'original {vitals_names[j]} vitals array shape: {np.array(temp_vitals[j]).shape} ')
            for i in range(len(vitals_SBS)): #go through every SBS score for each vitals metric
                if (Filtering.checkVitals(temp_vitals[j][i], slice_size_min, vitals_names[j])): #check the data in a single window
                    vitals_list[j].append(temp_vitals[j][i]) #append that single window data to the 2D hr,rr,spo2,bpm,bps,bpd arrays if that window's data is valid
                else:
                    vitals_list[j].append(flag_list) #append an array of zeros for window number i for the jth vitals metric if the data is invalid(i.e. too many NaN points)
                    print(f'{vitals_names[j]} SBS index {i} has insufficient data, zeros appended in place') 
            print(f'final {vitals_names[j]} vitals array shape: {np.array(vitals_list[j]).shape}') #should be the number of SBS scores by the number of samples in a window


        j = 0

        for k in range(len(vitals_list)): 
            
            SBS_Changes = []
            AUC_Changes = []
            PEAK_PEAK_Changes = []
            Mean_Changes = []
            cleared_SBS = []
            AUC = []
            PEAK_PEAK = []
            Mean = []

            print(f'Calculating {vitals_names[k]} information for {patient}\n')
            signal_2D = (np.array(vitals_list[k])) #signal_2D is a single vitals metric

            for j in range(len(signal_2D)):
                if(np.mean(signal_2D[j])!=0): #check that neither is a flag list, signal_2D[j] is a single window for a single metric
                #define arrays to populate on each iteration
                    # Populate array with changes in SBS
                    cleared_SBS.append(vitals_SBS[j])
                    fillArrays(signal_2D[j], Mean, AUC, PEAK_PEAK)
                else:
                    print(f'flag list detected at index {j}')

            i = 0
            print(cleared_SBS)
            
            assert(len(Mean)== len(AUC))
            assert(len(AUC) == len(PEAK_PEAK))
            assert(len(cleared_SBS)==len(Mean))
            assert(len(vitals_SBS) >= len(cleared_SBS))

            count_mean = 0
            count_peak = 0
            count_AUC = 0
    
            factor = (1/20)

            if(len(cleared_SBS) > 3):
                q3, q1 = np.percentile(np.array(Mean), [75 ,25])
                iqr = q3 - q1
                threshold = factor * iqr
                print(f'THRESHOLD FOR MEAN: {threshold}')
                q3, q1 = np.percentile(np.array(PEAK_PEAK), [75 ,25])
                iqr = q3 - q1
                threshold1 = factor * iqr
                q3, q1 = np.percentile(np.array(AUC), [75 ,25])
                iqr = q3 - q1
                threshold2 = factor * iqr     

                for i in range(len(cleared_SBS)-1):
                    fill_SBS_Changes(cleared_SBS,SBS_Changes,i)

                    state = Mean[i+1] - Mean[i]
                    if(state > threshold):
                        Mean_Changes.append(1)
                    elif(state < -1*threshold): 
                        Mean_Changes.append(-1)
                    else:
                        Mean_Changes.append(0)
                    if(Mean_Changes[i] == SBS_Changes[i]):
                        count_mean+=1

                    state1 = PEAK_PEAK[i+1] - PEAK_PEAK[i]
                    if(state1 > threshold1):
                        PEAK_PEAK_Changes.append(1)
                    elif(state1< -1*threshold1): 
                        PEAK_PEAK_Changes.append(-1)
                    else:
                        PEAK_PEAK_Changes.append(0)
                    if(PEAK_PEAK_Changes[i] == SBS_Changes[i]):
                        count_peak+=1

                    state2 = AUC[i+1] - AUC[i]
                    if(state2 > threshold2):
                        AUC_Changes.append(1)
                    elif state2 < -1*threshold2: 
                        AUC_Changes.append(-1)
                    else:
                        AUC_Changes.append(0)
                    if(AUC_Changes[i] == SBS_Changes[i]):
                        count_AUC+=1

                print(f'The below stats are based on {len(cleared_SBS)} SBS scores\n')
                print(f"Percentage of same changes with AUC is: {100*(count_AUC/len(SBS_Changes))}%\n")
                print(f"Percentage of same changes with Peak to Peak is: {100*(count_peak/len(SBS_Changes))}%\n")
                print(f"Percentage of same changes with Mean is: {100*(count_mean/len(SBS_Changes))}%\n")
            else:
                print('not enough SBS scores')




FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\sidha\\OneDrive\\Sid Stuff\\PROJECTS\\iMEDS Design Team\\Data Analysis\\PedAccel\\data_analysis\\PythonPipeline\\PatientData\\2_neg2\\2_neg2_SICKBAY10MIN_5MIN.mat'

In [9]:
#Patients with -2 or 2 are ['Patient13', 'Patient2', 'Patient3', 'Patient4', 'Patient9']
# data_dir = r'C:\Users\jakes\Documents\DT 6 Analysis\PythonCode\PedAccel\data_analysis\PythonPipeline\PatientData\2_neg2'
data_dir = r'C:\Users\sidha\OneDrive\Sid Stuff\PROJECTS\iMEDS Design Team\Data Analysis\PedAccel\data_analysis\PythonPipeline\PatientData\2_neg2'

window_size = 100 #100 is 1 second worth of time
lead_time = 10
slice_size_min = 15
sr = .5

In [10]:

#There is no error handling in place, the .mat file must exist
for patient in os.listdir(data_dir):
    # filter out non-directories
    patient_dir = os.path.join(data_dir, patient)
    if os.path.isdir(patient_dir):
       # data_filepath_accel = os.path.join(patient_dir, f'{patient}_{lead_time}MIN_{slice_size_min - lead_time}MIN.mat')
        data_filepath_vitals = os.path.join(patient_dir, f'{patient}_SICKBAY_{lead_time}MIN_{slice_size_min - lead_time}MIN.mat')
        # data_filepath_vitals = os.path.join(patient_dir, f'{patient}_SICKBAY_{slice_size_min - lead_time}MIN_{lead_time}MIN.mat')
        
       # accel_data = loadmat(data_filepath_accel)
       # x_mag = accel_data["x_mag"]
       # accel_SBS = accel_data["sbs"].flatten()
        
        vitals_data = loadmat(data_filepath_vitals)
        temp_hr = vitals_data['heart_rate']
        temp_SpO2 = vitals_data['SpO2']
        temp_rr = vitals_data['respiratory_rate']

        vitals_SBS = vitals_data['sbs'].flatten()
        hr = []
        rr = []
        SpO2 = []
        bpm = []
        bps = []
        bpd = []
        vitals_list = [hr,rr,SpO2]
        vitals_names = ['hr','rr','spo2']
        temp_vitals = [temp_hr,temp_rr, temp_SpO2] 
        
        flag_list = [0] * (int)(sr * 60 * slice_size_min) #generate a list to insert in place of invalid data, 
        #this list serves as a flag for a window to ignore in the box plot function

        for j in range(len(vitals_list)): #go through every vitals metric
            #print(f'original {vitals_names[j]} vitals array shape: {np.array(temp_vitals[j]).shape} ')
            for i in range(len(vitals_SBS)): #go through every SBS score for each vitals metric
                if (Filtering.checkVitals(temp_vitals[j][i], slice_size_min, vitals_names[j])): #check the data in a single window
                    vitals_list[j].append(temp_vitals[j][i]) #append that single window data to the 2D hr,rr,spo2,bpm,bps,bpd arrays if that window's data is valid
                else:
                    vitals_list[j].append(flag_list) #append an array of zeros for window number i for the jth vitals metric if the data is invalid(i.e. too many NaN points)
                   # print(f'{vitals_names[j]} SBS index {i} has insufficient data, zeros appended in place') 
            #print(f'final {vitals_names[j]} vitals array shape: {np.array(vitals_list[j]).shape}') #should be the number of SBS scores by the number of samples in a window


        j = 0

        for k in  range(len(vitals_list)): 
            
            SBS_Changes = []
            AUC_Changes = []
            PEAK_PEAK_Changes = []
            Mean_Changes = []
            cleared_SBS = []
            AUC = []
            PEAK_PEAK = []
            Mean = []

            signal_2D = (np.array(vitals_list[k])) #signal_2D is a single vitals metric

            for j in range(len(signal_2D)):
                if(np.mean(signal_2D[j])!=0): #check that neither is a flag list, signal_2D[j] is a single window for a single metric
                #define arrays to populate on each iteration
                    # Populate array with changes in SBS
                    cleared_SBS.append(vitals_SBS[j])
                    fillArrays(signal_2D[j], Mean, AUC, PEAK_PEAK)
                #else:
                    #print(f'flag list detected at index {j}')

            i = 0
            
            assert(len(Mean)== len(AUC))
            assert(len(AUC) == len(PEAK_PEAK))
            assert(len(cleared_SBS)==len(Mean))
            assert(len(vitals_SBS) >= len(cleared_SBS))


            average_change_in_mean_2 = []
            average_change_in_AUC_2 = []
            average_change_in_Peaks_2 = []

            average_change_in_mean_0 = []
            average_change_in_AUC_0 = []
            average_change_in_Peaks_0 = []

            if(len(cleared_SBS) > 2):
                for i in range(len(cleared_SBS)-1):
                    if((cleared_SBS[i] == 2) and cleared_SBS[i+1] != 2):
                        print(f'The SBS score at index {i} is {cleared_SBS[i]} and the SBS score at index {i + 1} is {cleared_SBS[i+1]}')
                        print(f'The change in PEAK_PEAK values from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {PEAK_PEAK[i]-PEAK_PEAK[i+1]}')
                        print(f'The change in AUC from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {AUC[i]-AUC[i+1]}')
                        print(f'The change in Mean values from {cleared_SBS[i]} to {cleared_SBS[i+1]} is {Mean[i]-Mean[i+1]}')
                        average_change_in_mean_2.append(abs(Mean[i]-Mean[i+1]))
                        average_change_in_AUC_2.append(abs(AUC[i]-AUC[i+1]))
                        average_change_in_Peaks_2.append(abs(PEAK_PEAK[i]-PEAK_PEAK[i+1]))

                    elif((cleared_SBS[i] == 0) and (cleared_SBS[i+1] == 1 or cleared_SBS[i+1] == -1)):
                        average_change_in_mean_0.append(abs(Mean[i]-Mean[i+1]))
                        average_change_in_AUC_0.append(abs(AUC[i]-AUC[i+1]))
                        average_change_in_Peaks_0.append(abs(PEAK_PEAK[i]-PEAK_PEAK[i+1]))
                print('_____________________________________________________________________________________________________')
                print(f'******** SUMMARY {vitals_names[k]} {patient}*********\n')
                print(f'The average change in mean from SBS 2 to any other scores was {np.mean(average_change_in_mean_2)}')
                print(f'The average change in AUC from SBS 2 to any other scores was {np.mean(average_change_in_AUC_2)}')
                print(f'The average change in Peak to Peak from SBS 2 to any other scores was {np.mean(average_change_in_Peaks_2)}')
                print(f'The average change in mean from SBS 0 to -1 or 1 was {np.mean(average_change_in_mean_0)}')
                print(f'The average change in AUC from SBS 0 to -1 or 1 was {np.mean(average_change_in_AUC_0)}')
                print(f'The average change in Peak to Peak from SBS 0 to -1 or 1 was {np.mean(average_change_in_Peaks_0)}')
                print('_____________________________________________________________________________________________________')



The SBS score at index 11 is 2 and the SBS score at index 12 is -1
The change in PEAK_PEAK values from 2 to -1 is 46.0
The change in AUC from 2 to -1 is 28526.0
The change in Mean values from 2 to -1 is 63.44
The SBS score at index 13 is 2 and the SBS score at index 14 is 1
The change in PEAK_PEAK values from 2 to 1 is 3.0
The change in AUC from 2 to 1 is 16719.5
The change in Mean values from 2 to 1 is 37.24444444444444
The SBS score at index 17 is 2 and the SBS score at index 18 is -1
The change in PEAK_PEAK values from 2 to -1 is 62.0
The change in AUC from 2 to -1 is 8198.5
The change in Mean values from 2 to -1 is 18.186666666666667
The SBS score at index 19 is 2 and the SBS score at index 20 is 0
The change in PEAK_PEAK values from 2 to 0 is 30.0
The change in AUC from 2 to 0 is 31074.5
The change in Mean values from 2 to 0 is 69.16222222222221
The SBS score at index 21 is 2 and the SBS score at index 22 is -1
The change in PEAK_PEAK values from 2 to -1 is -3.0
The change in AUC 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\sidha\\OneDrive\\Sid Stuff\\PROJECTS\\iMEDS Design Team\\Data Analysis\\PedAccel\\data_analysis\\PythonPipeline\\PatientData\\2_neg2\\Patient3\\Patient3_SICKBAY_10MIN_5MIN.mat'